# Spark Streaming with RDD

In [0]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('Streaming')
sc = SparkContext.getOrCreate(conf=conf)

ssc = StreamingContext.getActiveOrCreate(sc, 1) # 1 for time interwell

In [0]:
rdd = ssc.textFileStream('/FileStore/tables')
rdd.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(100)

-------------------------------------------
Time: 2022-05-02 12:26:22
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:23
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:24
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:25
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:26
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:27
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:28
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:29
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:26:30
----------

# Spark Streaming Reading Data

In [0]:
ssc.stop()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-79936875505568> in <module>
----> 1 ssc.stop()

NameError: name 'ssc' is not defined

In [0]:
rdd = ssc.textFileStream('/FileStore/tables')

In [0]:
rdd.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(100)

-------------------------------------------
Time: 2022-05-02 12:32:28
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:29
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:30
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:31
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:32
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:33
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:34
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:35
-------------------------------------------

-------------------------------------------
Time: 2022-05-02 12:32:36
----------

# Spark Streaming RDD Transformations

In [0]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('Streaming')
sc = SparkContext.getOrCreate(conf=conf)

ssc = StreamingContext(sc, 1)

In [0]:
dbutils.fs.rm('/FileStore/tables', True)

rdd = ssc.textFileStream('/FileStore/tables')

In [0]:
rdd1 = rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: (x + y))
rdd1.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(50)

-------------------------------------------
Time: 2022-05-05 16:40:25
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:26
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:27
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:28
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:29
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:30
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:31
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:32
-------------------------------------------

-------------------------------------------
Time: 2022-05-05 16:40:33
----------

In [0]:
# here in RDD if we perform transformations on streaming data reasult of first transformation will not merge will secont transformation.. they both remains independent.. 
# the only diff working with regular spark and Spark Streaming are- in regular spark we will get the data from regular file in batch manner, in Spart Streaming we will get the data in streaming manner

# Both SparkContext and StreamingContext are required to create spark stream


# Spark Streaming DF

In [0]:
dbutils.fs.rm('FileStore/tables', True)

Out[5]: True

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Streaming').getOrCreate()

word = spark.readStream.text('/FileStore/tables')

In [0]:
word.writeStream.format("console").outputMode("complete").start()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-748909809404635> in <module>
----> 1 word.writeStream.format("console").outputMode("complete").start()

/databricks/spark/python/pyspark/sql/streaming.py in start(self, path, format, outputMode, partitionBy, queryName, **options)
   1223             self.queryName(queryName)
   1224         if path is None:
-> 1225             return self._sq(self._jwrite.start())
   1226         else:
   1227             return self._sq(self._jwrite.start(path))

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121        

In [0]:
display(word)

value
Apple
Mic
Mic
Apple
Laptop
Apple
Mic
Mic
Apple
Laptop


# Spark Streaming DF Aggregations

In [0]:
word = word.groupBy('value').count()
display(word)

value,count
Mic,10
Chair,15
Book,5
Laptop,5
Bag,5
Mobile,5
Apple,10


In [0]:
word = word.groupBy('value').count()
display(word)

value,count
Mic,20
Chair,30
Book,10
Laptop,10
Bag,10
Mobile,10
Apple,20


In [0]:
# streaming RDD will not consider already existing files while transforming
# streaming DF will consider all existing files while transforming